In [ ]:
import pandas as pd
import numpy as py

In [ ]:
fama3=pd.read_csv('Fama.csv')
benchmark=pd.read_csv('Risk Adj Return.csv')
xgb=pd.read_csv('proba/proba_XG2.csv')
rf=pd.read_csv('proba/proba_RF2.csv')
lr=pd.read_csv('proba/proba_LR2.csv')
ann=pd.read_csv('proba/proba_ANN2.csv')

In [ ]:
benchmark=benchmark.iloc[453:-197,:]#216-general trimmming, 240 volatility trimming,247 7 days pred trimming, 499 predic

In [ ]:
date=pd.to_datetime(benchmark['Date'],format='%d/%m/%Y')
date_ANN=pd.to_datetime(benchmark['Date'])[252:]

1. Check Date for Results

In [ ]:
a=[]
for i in range(90):
    a.append(date[252*i:252*(i+1)])

2. 

In [ ]:
benchmark=benchmark.iloc[:,1:].astype(float)

In [ ]:
ANN_bm=benchmark.iloc[252:,:].copy()

In [ ]:
xgb=xgb.iloc[:,1:].dropna()
rf=rf.iloc[:,1:].dropna()
lr=lr.iloc[:,1:].dropna()
ann=ann.iloc[:,1:].dropna()

In [ ]:
RF=[]
XGB=[]
LR=[]
ANN=[]
for i in range(93):
    XGB.append(py.concatenate(py.array(xgb.iloc[:,42*i:42*(i+1)]),axis=None))
    RF.append(py.concatenate(py.array(rf.iloc[:,42*i:42*(i+1)]),axis=None))
    LR.append(py.concatenate(py.array(lr.iloc[:,42*i:42*(i+1)]),axis=None))
    ANN.append(py.concatenate(py.array(ann.iloc[:,42*i:42*(i+1)]),axis=None))

In [ ]:
RF=pd.DataFrame(RF).astype(float).T
XGB=pd.DataFrame(XGB).astype(float).T
LR=pd.DataFrame(LR).astype(float).T
Combined=(RF+XGB+LR)/3
ANN=pd.DataFrame(ANN).astype(float).T

1. Prediction Visualization 

In [ ]:
correl=pd.DataFrame(list(zip(py.concatenate(RF.to_numpy()),py.concatenate(XGB.to_numpy()),py.concatenate(LR.to_numpy()))))
correl.corr()

In [ ]:
import seaborn as sns 
import matplotlib.pyplot as pl
kwargs=dict(kde_kws={'linewidth':2})
a=pd.DataFrame(list(zip(py.concatenate(RF.to_numpy()),py.concatenate(XGB.to_numpy()),py.concatenate(LR.to_numpy()))))
pl.figure(figsize=(10,7),dpi=80)

sns.displot(a.iloc[:,0],color="dodgerblue",label="RF",**kwargs)
sns.displot(a.iloc[:,1],color="orange",label="XGB",**kwargs)
sns.displot(a.iloc[:,2],color="deeppink",label="LR",**kwargs)
sns.displot(py.concatenate(ANN.to_numpy()),color="black",label="ANN",**kwargs)
#sns.displot(py.concatenate(Combined.to_numpy()),color="deeppink",label="Combined",**kwargs)

pl.show()

2. Portfolio

In [ ]:
def long_short_ret(model,benchmark): 
    ret_day=[]
    for i in range(model.shape[0]):
        stock_ret_day=[]
        for j in range(model.shape[1]):
            if model.iloc[i,j] >= py.percentile(model.iloc[i,:],90):
                stock_ret_day.append(benchmark.iloc[i,j])
            #elif model.iloc[i,j] <= py.percentile(model.iloc[i,:],):
             #   stock_ret_day.append(-benchmark.iloc[i,j])
            else:
                pass
        ret_day.append(py.sum(stock_ret_day))
    return py.mean(ret_day),ret_day,py.cumsum(ret_day)

In [ ]:
RF_mean,RF_ret_day,RF_cumsum=long_short_ret(RF,benchmark)
XG_mean,XG_ret_day,XG_cumsum=long_short_ret(XGB,benchmark)
LR_mean,LR_ret_day,LR_cumsum=long_short_ret(LR,benchmark)
Comb_mean,Comb_ret_day,Comb_cumsum=long_short_ret(Combined,benchmark)
ANN_mean,ANN_ret_day,ANN_cumsum=long_short_ret(ANN,ANN_bm)

In [ ]:
pd.DataFrame(list(zip(date,RF_ret_day,XG_ret_day,LR_ret_day,Comb_ret_day,RF_cumsum,XG_cumsum,LR_cumsum,Comb_cumsum))).to_csv('portfolioret2.csv')

In [ ]:
pd.DataFrame(list(zip(date_ANN,ANN_ret_day,ANN_cumsum))).to_csv('ANNportfolioret.csv')

In [ ]:
ANN_cumsum

Fama-French Regression

In [ ]:
import statsmodels.api as sm

In [ ]:
fama3=fama3.iloc[:,:-1]
fama3['Date']=pd.to_datetime(fama3.iloc[:,0],format="%d/%m/%Y")

In [ ]:
def fama_reg(ret,date):
    a=pd.DataFrame(ret,index=date)
    fama=fama3.merge(a,on='Date')
    X=fama[['Mkt-RF','SMB','HML','RMW','CMA']]
    X_sm=sm.add_constant(X)
    model=sm.OLS(fama[0],X_sm)
    results=model.fit()
    return results.summary()

In [ ]:
fama_reg(ANN_ret_day,date_ANN)